In [8]:
import torch
from torchaudio.transforms import MelSpectrogram
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
from torchaudio.transforms import InverseMelScale, GriffinLim

device = "cuda" if torch.cuda.is_available() else "cpu"


def preprocess_wav(file_path, model):
    waveform, sample_rate = torchaudio.load(file_path)
    n_fft = 2048  # Higher FFT size for better frequency resolution
    hop_length = 64  # Smaller hop length for better temporal resolution
    n_mels = 64  # Higher mel bands for better frequency resolution
    
    mel_transform = MelSpectrogram(sample_rate=sample_rate, n_mels=n_mels, hop_length=hop_length, n_fft=n_fft)
    mel_spectrogram = mel_transform(waveform)
    
    mel_spectrogram = mel_spectrogram.unsqueeze(0).to(device)
    
    with torch.no_grad():
        latent_representation = model.encoder(mel_spectrogram)
    return latent_representation


def apply_gain(waveform, gain_db):
    gain = 10 ** (gain_db / 20)
    return waveform * gain

def compress_dynamic_range(waveform, threshold=-20.0, ratio=5.0):
    db_waveform = 20 * torch.log10(waveform.abs() + 1e-10)
    db_waveform = torch.where(db_waveform > threshold,
                              threshold + (db_waveform - threshold) / ratio,
                              db_waveform)
    compressed_waveform = 10 ** (db_waveform / 20)
    return compressed_waveform



class Encoder(nn.Module):
    def __init__(self, in_channels, hidden_channels, n_embeddings, embedding_dim):
        super(Encoder, self).__init__()
        self.dropout = nn.Dropout(p=0.2)
        self.conv1 = nn.Conv2d(in_channels, hidden_channels, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.Conv2d(hidden_channels, hidden_channels, kernel_size=4, stride=2, padding=1)
        self.conv3 = nn.Conv2d(hidden_channels, embedding_dim, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        # x = self.dropout(F.relu(self.conv1(x)))
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.conv3(x)
        return x


class Decoder(nn.Module):
    def __init__(self, embedding_dim, hidden_channels, out_channels):
        super(Decoder, self).__init__()
        self.dropout = nn.Dropout(p=0.2)
        self.conv1 = nn.ConvTranspose2d(embedding_dim, hidden_channels, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.ConvTranspose2d(hidden_channels, hidden_channels, kernel_size=4, stride=2, padding=1)
        self.conv3 = nn.ConvTranspose2d(hidden_channels, out_channels, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        # x = self.dropout(F.relu(self.conv1(x)))
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = torch.sigmoid(self.conv3(x))
        return x


class VectorQuantizer(nn.Module):
    def __init__(self, n_embeddings, embedding_dim, commitment_cost=0.25):
        super(VectorQuantizer, self).__init__()
        self.embedding_dim = embedding_dim
        self.n_embeddings = n_embeddings
        self.embeddings = nn.Embedding(n_embeddings, embedding_dim)
        self.embeddings.weight.data.uniform_(-1 / n_embeddings, 1 / n_embeddings)
        self.commitment_cost = commitment_cost

    def forward(self, inputs):
        flat_input = inputs.view(-1, self.embedding_dim)
        distances = (torch.sum(flat_input ** 2, dim=1, keepdim=True)
                     + torch.sum(self.embeddings.weight ** 2, dim=1)
                     - 2 * torch.matmul(flat_input, self.embeddings.weight.t()))

        encoding_indices = torch.argmin(distances, dim=1).unsqueeze(1)
        quantized = torch.index_select(self.embeddings.weight, 0, encoding_indices.view(-1))
        quantized = quantized.view(inputs.shape)

        # Commitment Loss
        e_latent_loss = F.mse_loss(quantized.detach(), inputs)
        q_latent_loss = F.mse_loss(quantized, inputs.detach())
        loss = q_latent_loss + self.commitment_cost * e_latent_loss

        quantized = inputs + (quantized - inputs).detach()
        return quantized, loss, encoding_indices


class VQVAE2(nn.Module):
    def __init__(self, in_channels=1, hidden_channels=64, n_embeddings=512, embedding_dim=64):
        super(VQVAE2, self).__init__()
        self.encoder = Encoder(in_channels, hidden_channels, n_embeddings, embedding_dim)
        self.vq = VectorQuantizer(n_embeddings, embedding_dim)
        self.decoder = Decoder(embedding_dim, hidden_channels, in_channels)

    def forward(self, x):
        z = self.encoder(x)
        quantized, vq_loss, _ = self.vq(z)
        x_recon = self.decoder(quantized)
        x_recon = F.interpolate(x_recon, size=x.shape[-2:], mode='bilinear', align_corners=False)
        return x_recon, vq_loss


from torch.utils.data import Dataset, DataLoader
import os


class WavDataset(Dataset):
    def __init__(self, file_dir, model):
        self.file_dir = file_dir
        self.files = [os.path.join(file_dir, f) for f in os.listdir(file_dir) if f.endswith('.wav')]
        self.model = model

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path = self.files[idx]
        latent_representation = preprocess_wav(file_path, self.model)
        return latent_representation



def pad_collate(batch):
    max_len = max([x.shape[-1] for x in batch])
    batch = [F.pad(x, (0, max_len - x.shape[-1])) for x in batch]
    return torch.stack(batch)

def train(model, data_loader, epochs=150, lr=1e-3):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.L1Loss()

    for epoch in range(epochs):
        total_loss = 0
        for latent_batch in data_loader:
            latent_batch = latent_batch.to(device)

            quantized, vq_loss, _ = model.vq(latent_batch)
            recon = model.decoder(quantized.squeeze())
            recon = F.interpolate(recon, size=latent_batch.shape[-2:], mode='bilinear', align_corners=False)
            
            recon_loss = criterion(recon, latent_batch)
            loss = recon_loss + 0.5 * vq_loss
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f'Epoch {epoch + 1}, Loss: {total_loss / len(data_loader)}')
        
        
model = VQVAE2(in_channels=1).to(device)

dataset = WavDataset(file_dir='../unpacked_data', model=model)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=pad_collate)

train(model, data_loader, epochs=2)


def generate_samples(model, num_samples=1, latent_shape=(64, 80), device=device):
    model.eval()
    with torch.no_grad():
        latent_code = torch.randn(num_samples, model.vq.embedding_dim, latent_shape[0] // 4, latent_shape[1] // 4).to(device)
        generated_spectrogram = model.decoder(latent_code)
        generated_spectrogram = generated_spectrogram.squeeze(1)
        return generated_spectrogram



def spectrogram_to_waveform(mel_spectrogram, sample_rate=16000, n_fft=2048, n_mels=64):
    inverse_mel_scale = InverseMelScale(n_stft=n_fft // 2 + 1, n_mels=n_mels).to(mel_spectrogram.device)
    linear_spectrogram = inverse_mel_scale(mel_spectrogram)
    # griffin_lim = GriffinLim(n_fft=n_fft).to(mel_spectrogram.device)
    griffin_lim = GriffinLim(n_fft=n_fft, n_iter=100).to(mel_spectrogram.device)  # More iterations
    waveform = griffin_lim(linear_spectrogram)
    return waveform


def save_generated_audio(model, file_name="generated_sample.wav", device="cuda"):
    generated_mel_spectrogram = generate_samples(model, num_samples=1, device=device)
    generated_waveform = spectrogram_to_waveform(generated_mel_spectrogram, sample_rate=16000)
    generated_waveform = generated_waveform.cpu()
    generated_waveform = generated_waveform.unsqueeze(0) if generated_waveform.ndim == 1 else generated_waveform
    generated_waveform = apply_gain(generated_waveform, 40)
    torchaudio.save(file_name, generated_waveform, 16000)


save_generated_audio(model, file_name="generated_sample.wav", device=device)


Epoch 1, Loss: 88.7220064798991


C:\Users\shale\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:101: UserWarning: Using a target size (torch.Size([32, 1, 64, 16, 492])) that is different to the input size (torch.Size([32, 1, 16, 492])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Epoch 2, Loss: 91.22310002644856
